reulst_summary_lstm :
decimal point, time in hh.mm.ss
remove normalization and make one single table.

TODO::
Do for all 8 datasets. First the even 4, then odd 4. change the time() placement. 
Save actual and predicted target for every model.

forward feature selection:
model for first 3 ( i made)
model for those 3 + iteratively add one from the given subset
model for all the above
model for all the above + ctrl_diff2|1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.decomposition import PCA

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
file_path = ['C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_AL_CP1.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_AL_CP2.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_S_CP1.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/1_CMX/CMX1_S_CP2.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_AL_CP1.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_AL_CP2.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_S_CP1.csv',
             'C:/Users/genty/Hiwi-TimeSeries/dataset/DatAmount_dataset_new/2_DMC/DMC2_S_CP2.csv']

In [5]:
features =  ['LOAD|1', 'LOAD|2', 'LOAD|3', 'LOAD|6',
       'ENC_POS|1', 'ENC_POS|2', 'ENC_POS|3','ENC_POS|6',
       'CTRL_DIFF2|1', 'CTRL_DIFF2|2', 'CTRL_DIFF2|3', 'CTRL_DIFF2|6',
       'TORQUE|1', 'TORQUE|2', 'TORQUE|3', 'TORQUE|6',
       'DES_POS|1', 'DES_POS|2', 'DES_POS|3', 'DES_POS|6',
       'CTRL_DIFF|1', 'CTRL_DIFF|2', 'CTRL_DIFF|3' ,'CTRL_DIFF|6',
       'CTRL_POS|1', 'CTRL_POS|2', 'CTRL_POS|3', 'CTRL_POS|6',
       'VEL_FFW|1', 'VEL_FFW|2','VEL_FFW|3', 'VEL_FFW|6',
       'CONT_DEV|1','CONT_DEV|2', 'CONT_DEV|3', 'CONT_DEV|6',
       'CMD_SPEED|1', 'CMD_SPEED|2', 'CMD_SPEED|3', 'CMD_SPEED|6',
       'TORQUE_FFW|1', 'TORQUE_FFW|2', 'TORQUE_FFW|3', 'TORQUE_FFW|6',
       'ENC1_POS|1', 'ENC1_POS|2', 'ENC1_POS|3','ENC1_POS|6',
       'ENC2_POS|1', 'ENC2_POS|2', 'ENC2_POS|3', 'ENC2_POS|6']

feature_subset = ['TORQUE|6', 'LOAD|6', 'CMD_SPEED|6', 'VEL_FFW|6',  'VEL_FFW|1', 'VEL_FFW|2']
#['TORQUE|6', 'LOAD|6', 'TORQUE_FFW|6', 'CTRL_DIFF2|1']
target = 'CURRENT|6'

In [6]:
data_split = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
feature_list = [feature_subset, features]

# RF

In [ ]:
output_folder = '/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/output/8dataset/'
#result_df = pd.DataFrame(columns=['Dataset', 'Data_Split', 'Features', 'Normalization', 'Train_R2', 'Train_RMSE', 'Test_R2', 'Test_RMSE'])
result_df = pd.read_csv('/home/genty/Hiwi-TimeSeries/Hiwi-TimeSeries/output/8dataset/result_summary_rf.csv')
filenamefinal = os.path.join(output_folder, f'result_summary_rf.csv')

In [ ]:
def normalize(X, y):
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    scaled_X = scaler_X.fit_transform(X)
    scaled_y = scaler_y.fit_transform(y)
    return scaler_X, scaler_y, scaled_X, scaled_y

In [ ]:
for file in file_path:
    df = pd.read_csv(file)
    for split_ratio in data_split:
        for f in feature_list:
            feature_type = 'all' if len(f) == len(features) else 'subset'
            for n in norm:
                X = df[f]
                y = df[[target]]
                rf_model = RandomForestRegressor()
                if n == 'yes':
                    scaler_X, scaler_y, scaled_X, scaled_y = normalize(X, y)
                    X = scaled_X
                    y = scaled_y
                train_size = int(len(X) * split_ratio)
                X_train, X_test = X[:train_size], X[train_size:]
                y_train, y_test = y[:train_size], y[train_size:]

                rf_model.fit(X_train, y_train)
                train_pred = rf_model.predict(X_train)
                test_pred = rf_model.predict(X_test)
                #print(y_pred)
                '''y_train = y_train.squeeze()
                y_test = y_test.squeeze()

                train_df = pd.DataFrame({
                        'Train_y': y_train,
                        'Train_pred_y': train_pred })
                filename = os.path.join(output_folder, f'trains_{os.path.basename(file)}_3070.csv')
                train_df.to_csv(filename, index=False)

                test_df = pd.DataFrame({
                        'Test_y': y_test,
                        'Test_pred_y': test_pred })
                filename = os.path.join(output_folder, f'tests_{os.path.basename(file)}_3070.csv')
                test_df.to_csv(filename, index=False)'''
                
                train_mse = mean_squared_error(y_train, train_pred)
                train_rmse = np.sqrt(train_mse)
                train_r2 = r2_score(y_train, train_pred)
                test_mse = mean_squared_error(y_test, test_pred)
                test_rmse = np.sqrt(test_mse)
                test_r2 = r2_score(y_test, test_pred)

                result_df = result_df.append({
                        'Dataset': os.path.basename(file),
                        'Data_Split': split_ratio,
                        'Features': feature_type,
                        'Normalization': n,
                        'Train_R2': train_r2,
                        'Train_RMSE': train_rmse,
                        'Test_R2': test_r2,
                        'Test_RMSE': test_rmse
                        }, ignore_index=True)

                result_df.to_csv(filenamefinal, index=False)
                print("Done")


/home/genty/.local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Done


/home/genty/.local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Done


/home/genty/.local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

Done


/home/genty/.local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Done


/home/genty/.local/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Done


# LSTM

In [2]:
output_folder = 'C:/Users/genty/Hiwi-TimeSeries/output/8dataset/'
#result_df = pd.DataFrame(columns=['Dataset', 'Data_Split', 'Features', 'RunTime', 'Train_R2', 'Train_RMSE', 'Test_R2', 'Test_RMSE'])
result_df = pd.read_csv('C:/Users/genty/Hiwi-TimeSeries/output/8dataset/result_summary_lstm_newest.csv')
filenamefinal = os.path.join(output_folder, f'result_summary_lstm_newest.csv')

In [8]:
def normalize(X, y):
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    scaled_X = scaler_X.fit_transform(X)
    scaled_y = scaler_y.fit_transform(y)

    return scaler_X, scaler_y, scaled_X, scaled_y

In [9]:
def denormalize(train_predict, y_train, test_predict, y_test, scaler_y):
    train_predict_inv = scaler_y.inverse_transform(train_predict)
    y_train_inv = scaler_y.inverse_transform(y_train)
    test_predict_inv = scaler_y.inverse_transform(test_predict)
    y_test_inv = scaler_y.inverse_transform(y_test)
    
    return train_predict_inv, y_train_inv, test_predict_inv, y_test_inv

In [10]:
def create_dataset(X, y, time_step=60):
    Xs, ys = [], []
    for i in range(len(X) - time_step):
        if isinstance(X, np.ndarray):
            Xs.append(X[i:(i + time_step)])
            ys.append(y[i + time_step])
        else:
            Xs.append(X.iloc[i:(i + time_step)].values)
            ys.append(y.iloc[i + time_step].values)
    return np.array(Xs), np.array(ys)

In [11]:
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [13]:
for file in file_path[4:5:2]:
    df = pd.read_csv(file)
    for split_ratio in data_split:
        for f in feature_list:
                print(f'File-split-{split_ratio}-features{len(f)}')
                if ((split_ratio==0.3) and (len(f)==6)):
                        feature_type = 'all' if len(f) == len(features) else 'subset'
                        X = df[f]
                        y = df[[target]]
                        scaler_X, scaler_y, scaled_X, scaled_y = normalize(X, y)
                        X = scaled_X
                        y = scaled_y

                        train_size = int(len(X) * split_ratio)
                        test_size = len(X) - train_size
                        X_train, X_test = X[:train_size], X[train_size:]
                        y_train, y_test = y[:train_size], y[train_size:]

                        
                        time_step = 60
                        X_train, y_train = create_dataset(X_train, y_train, time_step)
                        X_test, y_test = create_dataset(X_test, y_test, time_step)

                        input_shape = (X_train.shape[1], X_train.shape[2])
                        model = build_model(input_shape)
                        start_time = time.time()
                        history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=0)
                        end_time = time.time()
                        train_predict = model.predict(X_train)
                        test_predict = model.predict(X_test)
                        train_predict, y_train, test_predict, y_test = denormalize(train_predict, y_train, test_predict, y_test, scaler_y)


                        train_df = pd.DataFrame({
                                'Train_y': y_train.flatten(),
                                'Train_pred_y': train_predict.flatten() })
                        filename = os.path.join(output_folder, f'trains_{os.path.basename(file)}_sp{split_ratio}_{feature_type}.csv')
                        train_df.to_csv(filename, index=False)

                        test_df = pd.DataFrame({
                                'Test_y': y_test.flatten(),
                                'Test_pred_y': test_predict.flatten() })
                        filename = os.path.join(output_folder, f'tests_{os.path.basename(file)}_sp{split_ratio}_{feature_type}.csv')
                        test_df.to_csv(filename, index=False)
                        
                        runtime = end_time - start_time
                        hours = int(runtime//3600)
                        minutes = int((runtime%3600) //60)
                        seconds = int(runtime%60)
                        runtime = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
                        train_r2 = r2_score(y_train, train_predict)
                        test_r2 = r2_score(y_test, test_predict)
                        train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
                        test_rmse = np.sqrt(mean_squared_error(y_test, test_predict))

                        result_df = pd.concat([result_df, pd.DataFrame([{
                                        'Dataset': os.path.basename(file),
                                        'Data_Split': split_ratio,
                                        'Features': feature_type,
                                        'RunTime': runtime,
                                        'Train_R2': train_r2,
                                        'Test_R2': test_r2,
                                        'Train_RMSE': train_rmse,
                                        'Test_RMSE': test_rmse
                                        }])], ignore_index=True)

                        result_df.to_csv(filenamefinal, index=False)
                

File-split-0.2-features6
File-split-0.2-features52
File-split-0.3-features6


C:\Users\genty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4780/4780 ━━━━━━━━━━━━━━━━━━━━ 52s 11ms/step
11156/11156 ━━━━━━━━━━━━━━━━━━━━ 122s 11ms/step
File-split-0.3-features52
File-split-0.4-features6
File-split-0.4-features52
File-split-0.5-features6
File-split-0.5-features52
File-split-0.6-features6
File-split-0.6-features52
File-split-0.7-features6
File-split-0.7-features52
File-split-0.8-features6
File-split-0.8-features52
